### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
purchase_csv = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_pd = pd.read_csv(purchase_csv)
purchase_pd.head()



,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
# store into Pandas data frame
heroes_df = pd.DataFrame(purchase_pd, columns=['SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'])
heroes_df.head()

,SN,Age,Gender,Item ID,Item Name,Price
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,Ithergue48,24,Male,92,Final Critic,4.88
3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
player_count = len(heroes_df["SN"].value_counts())
player_count = pd.DataFrame({"Total Players": player_count}, index=[0])
player_count

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# Number of Unique Items
uniq_items = len(heroes_df['Item ID'].value_counts())
#uniq_items = purchase_pd['Item Name'].nunique()
# Average Price
#avg_price = heroes_df['Price'].mean()
avg_price = purchase_pd['Price'].mean()
# Number of Purchases
#totno_purchases = heroes_df['Item Name'].count()
totno_purchases = purchase_pd['Item Name'].count()
# Revenue
#revenue = heroes_df['Price'].sum()
revenue= purchase_pd['Price'].sum()
# DataFrame
#purchasing_analysis = pd.DataFrame({"Number of Unique Items": [uniq_items], "Average Price": [avg_price],
                                   #"Total Purchases": [totno_purchases], "Total Revenue": [revenue]})

purchasing_analysis = pd.DataFrame(
        {'Number of Unique Items': [uniq_items],
         'Average Price': [avg_price],
         'Number of Purchases': [totno_purchases],
         'Total Revenue': [revenue],})

purchasing_analysis ['Average Price'] = purchasing_analysis ['Average Price'].map("$ {:,.2f}".format)
purchasing_analysis ['Total Revenue'] = purchasing_analysis ['Total Revenue'].map("$ {:,.2f}".format)

purchasing_analysis [['Number of Unique Items','Average Price', 'Number of Purchases', 'Total Revenue']]

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$ 3.05,780,"$ 2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
g_groupby = purchase_pd.groupby('Gender')['SN'].nunique().reset_index()
g_groupby['Percentage of Players'] = 100 * g_groupby['SN']/g_groupby['SN'].sum()
g_summary = g_groupby[['Gender', 'Percentage of Players','SN' ]].sort_values(['Percentage of Players'],ascending = False)
g_summary = g_summary.reset_index(drop=True)

g_summary['Percentage of Players'] = g_summary['Percentage of Players'].map("{:,.2f}".format)

g_demograph_summ = g_summary.set_index('Gender')
g_demograph_summ = g_demograph_summ.rename(columns = {'SN': 'Total Count'})
g_demograph_summ

,Percentage of Players,Total Count
Gender,,
Male,84.03,484
Female,14.06,81
Other / Non-Disclosed,1.91,11



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
gender_demograph = purchase_pd.groupby("Gender")["Price"].agg(['sum','mean','count']).sort_values(['sum'],ascending = False).reset_index()
gender_female = gender_demograph[['sum']].iloc[1,0]/g_summary[['SN']].iloc[1,0]
gender_male = gender_demograph[['sum']].iloc[0,0]/g_summary[['SN']].iloc[0,0]
gender_other = gender_demograph [['sum']].iloc[2,0]/g_summary[['SN']].iloc[2,0]

g_summary['Normalized Totals'] = [gender_female, gender_male, gender_other]

merged_g = pd.merge(g_summary, gender_demograph, on = 'Gender')
merged_gender_df = merged_g.rename(columns = {'count': 'Purchase Count', 
                                               'mean' : 'Average Purchase Price', 
                                               'sum' : 'Total Purchase Value' })
merged_gender_df['Average Purchase Price'] = merged_gender_df['Average Purchase Price'].map("$ {:,.2f}".format)
merged_gender_df['Total Purchase Value'] = merged_gender_df['Total Purchase Value'].map("$ {:,.2f}".format)
merged_gender_df['Normalized Totals'] = merged_gender_df['Normalized Totals'].map("$ {:,.2f}".format)

purchasing_analysis_g = merged_gender_df.set_index('Gender')
purchasing_analysis_g[['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Normalized Totals']]



,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,652,$ 3.02,"$ 1,967.64",$ 4.47
Female,113,$ 3.20,$ 361.94,$ 4.07
Other / Non-Disclosed,15,$ 3.35,$ 50.19,$ 4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [65]:
#est age bins
age_bins = [0,9,14,19,24,29,34,39,44]

#categorize using bins
age_groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40>']
purchase_pd["Age Groups"] = pd.cut(purchase_pd["Age"], age_bins, labels=age_groups)

#Calculate the numbers and percentages by age group
age_groupby = purchase_pd.groupby('Age Groups')['SN'].nunique().reset_index()
age_groupby['Percentage of Players'] = 100 * age_groupby['SN']/g_groupby['SN'].sum()
age_summ = age_groupby[['Age Groups', 'Percentage of Players','SN' ]].sort_values(['Age Groups'])
age_summ = age_summ.reset_index(drop=True)

#Create a summary data frame to hold the results
age_summ['Percentage of Players'] = age_summ['Percentage of Players'].map("{:,.2f}".format)
age_demograph_summary = age_summ.set_index('Age Groups')
age_demograph_summary = age_demograph_summary.rename(columns = {'SN': 'Total Count'}) 
age_demograph_summary


,Percentage of Players,Total Count
Age Groups,,
<10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40>,1.91,11


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [75]:
age_demographs = purchase_pd.groupby('Age Groups')['Price'].agg(['sum', 'mean', 'count']).reset_index()
age_demographs

age_n_0 = age_demographs[['sum']].iloc[0,0]/age_summ[['SN']].iloc[0,0]
age_n_1 = age_demographs[['sum']].iloc[1,0]/age_summ[['SN']].iloc[1,0]
age_n_2 = age_demographs[['sum']].iloc[2,0]/age_summ[['SN']].iloc[2,0]
age_n_3 = age_demographs[['sum']].iloc[3,0]/age_summ[['SN']].iloc[3,0]
age_n_4 = age_demographs[['sum']].iloc[4,0]/age_summ[['SN']].iloc[4,0]
age_n_5 = age_demographs[['sum']].iloc[5,0]/age_summ[['SN']].iloc[5,0]
age_n_6 = age_demographs[['sum']].iloc[6,0]/age_summ[['SN']].iloc[6,0]
age_n_7 = age_demographs[['sum']].iloc[7,0]/age_summ[['SN']].iloc[7,0]

age_summ = [age_n_0, age_n_1, age_n_2, age_n_3, age_n_4, age_n_5, age_n_6, age_n_7]
age_summ = age_summ.map("$ {:,.2f}".format)

combined_age = pd.merge(age_summ, age_demographs, on='Age Groups')
combined_age_df = combined_age.rename(columns = {'count': 'Purchase Count', 'mean' : 'Average Purchase Price' , 'sum' : 'Total Purchase Value' })
age_purch_analysis = combined_age_df.set_index('Age Groups')
age_purch_analysis[['Purchase Count' ,'Average Purchase Price', 'Total Purchase Value' , "Average Total Purchase Per Person"]]



AttributeError: 'list' object has no attribute 'map'

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [36]:
top_sn = purchase_pd.groupby(["SN"])

total_price_sn = top_sn.sum()["Price"]

avg_price_sn = top_sn.mean()["Price"]

no_sn = top_sn.count()["Price"]

top_df = pd.DataFrame({"Purchase Count": no_sn, 
                       "Average Purchase Price":avg_price_sn,
                      "Total Purchase Price":total_price_sn})
sorted_df = top_df.sort_values("Total Purchase Price", ascending=False)

sorted_df["Average Purchase Price"] = sorted_df["Average Purchase Price"].map("${:,.2f}".format) 
sorted_df["Total Purchase Price"] = sorted_df["Total Purchase Price"].map("${:,.2f}".format)

sorted_df = sorted_df[["Purchase Count", "Average Purchase Price", "Total Purchase Price"]]
sorted_df.head()


,Purchase Count,Average Purchase Price,Total Purchase Price
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [53]:
count_items = purchase_pd.groupby(['Item ID','Item Name'])['Price'].agg(['count','sum','mean']).sort_values(['count'], ascending= False).reset_index()  
top_pop_items = count_items[:5]

most_pop = count_items.rename(columns = {'count': 'Purchase Count', 'mean' : 'Item Price', 
                                                    'sum' : 'Total Purchase Value' })
most_pop['Item Price'] = most_pop['Item Price'].map("$ {:,.2f}".format)
most_pop['Total Purchase Value'] = most_pop['Total Purchase Value'].map("$ {:,.2f}".format)

most_pop_df = most_pop.set_index(['Item ID', 'Item Name'])
most_pop_df[['Purchase Count', 'Item Price', 'Total Purchase Value']]
most_pop_df.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 50.76,$ 4.23
145,Fiery Glass Crusader,9,$ 41.22,$ 4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$ 31.77,$ 3.53
82,Nirvana,9,$ 44.10,$ 4.90
19,"Pursuit, Cudgel of Necromancy",8,$ 8.16,$ 1.02


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [54]:
sum_items = purchase_pd.groupby(['Item ID','Item Name'])['Price'].agg(['count','sum','mean']).sort_values(['sum'], ascending= False).reset_index()
top_pop_items = sum_items[:5]

top_profit_items = top_pop_items.rename(columns = {'count': 'Purchase Count', 
                                                   'mean': 'Item Price', 
                                                    'sum': 'Total Purchase Value'})
top_profit_items['Item Price'] = top_profit_items['Item Price'].map("$ {:,.2f}".format)
top_profit_items['Total Purchase Value'] = top_profit_items['Total Purchase Value'].map("$ {:,.2f}".format)

top_profit_items_df = top_profit_items.set_index(['Item ID', 'Item Name'])
top_profit_items_df[['Purchase Count' ,'Item Price', 'Total Purchase Value']]         

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$ 4.23,$ 50.76
82,Nirvana,9,$ 4.90,$ 44.10
145,Fiery Glass Crusader,9,$ 4.58,$ 41.22
92,Final Critic,8,$ 4.88,$ 39.04
103,Singed Scalpel,8,$ 4.35,$ 34.80
